#### **CVRP Miller–Tucker–Zemlin**

In [148]:
from docplex.mp.model import Model
import random
import plotly.express as px
import plotly.graph_objects as go

**Conjuntos**

$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$

In [149]:
N = 20
CLIENTES = list(range(1, N + 1))
NODOS = [0] + CLIENTES

**Parámetros**

$$ \text{CAPACIDAD\_CAMION} = 70 $$
$$ \text{DEMANDA}_{c}: \text{Demanda del cliente } c \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$


In [150]:
random.seed(142)

In [151]:
CAPACIDAD_CAMION = 70
DEMANDAS = {c: random.randint(10, 20) for c in CLIENTES}
COORDENADAS = {n: {"x": random.randint(0, N * 10), "y": random.randint(0, N * 10)} for n in NODOS}
DISTANCIAS = {(n1, n2): ((COORDENADAS[n1]["x"] - COORDENADAS[n2]["x"]) ** 2 + (COORDENADAS[n1]["y"] - COORDENADAS[n2]["y"]) ** 2) ** 0.5 for n1 in NODOS for n2 in NODOS}

In [152]:
fig = go.Figure()

# Añadir punto de depósito en rojo
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# Añadir puntos de clientes en negro
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

# Añadir anotaciones de clientes
for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}: </b> {DEMANDAS[cliente]}", showarrow=False)

# Configurar el layout del gráfico
fig.update_layout(
    title=f"COORDENADAS NODOS",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

In [153]:
model = Model("CVRP")

**Variables**

$$ x_{n1, n2} = \begin{cases} 
1 & \text{Si el camión viaja de } n1 \in \text{NODOS} \text{ al } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$
$$ u_{n} \in \mathbb{Z}^{+} \text{: Carga acumulada del vehículo al llegar al nodo } n \in \text{NODOS} $$

In [154]:
x = model.binary_var_dict([(nodo1, nodo2) for nodo1 in NODOS for nodo2 in NODOS], name="x")
u = model.integer_var_dict(NODOS, name="u")

**Función Objetivo**

$$ \text{Minimizar } \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1,n2} \cdot x_{n1, n2} $$

In [155]:
model.minimize(model.sum(DISTANCIAS[(nodo1, nodo2)] * x[(nodo1, nodo2)] for nodo1 in NODOS for nodo2 in NODOS))

**Restricciones**

1. **Prohibición de bucles:**
$$ \forall n \in \text{NODOS}: x_{n, n} = 0 $$

2. **Asignación única de salidas para cada cliente:**
$$ \forall c \in \text{CLIENTES}: \sum_{n \in \text{NODOS}, n \neq c} x_{c, n} = 1 $$

3. **Asignación única de entradas para cada cliente:**
$$ \forall c \in \text{CLIENTES}: \sum_{n \in \text{NODOS}, n \neq c} x_{n, c} = 1 $$

4. **Restricciones para evitar subtours:**
$$ \forall c1, c2 \in \text{CLIENTES}, c1 \neq c2: \text{Si } x_{c1, c2} = 1 \rightarrow u_{c2} =  u_{c1} + \text{DEMANDA}_{c2} $$

$$ \begin{cases} 
\forall c1, c2 \in \text{CLIENTES}, c1 \neq c2:  u_{c2} \leq u_{c1} + \text{DEMANDA}_{c2} + \mathcal{M} (1 - x_{c1, c2}) ❓\\
\forall c1, c2 \in \text{CLIENTES}, c1 \neq c2:  u_{c2} \geq u_{c1} + \text{DEMANDA}_{c2} - \mathcal{M} (1 - x_{c1, c2}) ✅
\end{cases} $$

5. **Demanda mínima en cada cliente:**
$$ \forall c \in \text{CLIENTES}: u_{c} \geq \text{DEMANDA}_{c} $$

6. **Demanda máxima permitida por el camión:**
$$ \forall c \in \text{CLIENTES}: u_{c} \leq \text{CAPACIDAD\_CAMION} $$


In [156]:
for nodo in NODOS:
    model.add_constraint(x[(nodo, nodo)] == 0)

for cliente in CLIENTES:
    model.add_constraint(model.sum(x[(cliente, nodo)] for nodo in NODOS if nodo != cliente) == 1)

for cliente in CLIENTES:
    model.add_constraint(model.sum(x[(nodo, cliente)] for nodo in NODOS if nodo != cliente) == 1)

for cliente1 in CLIENTES:
    for cliente2 in CLIENTES:
        if cliente1 != cliente2:
            # model.add_constraint(u[cliente2] >= u[cliente1] + DEMANDAS[cliente2] - 10000 * (1 - x[cliente1, cliente2]))
            model.add_indicator(x[(cliente1, cliente2)], u[cliente2] >= u[cliente1] + DEMANDAS[cliente2], active_value = 1)

for cliente in CLIENTES:
    model.add_constraint(u[cliente] >= DEMANDAS[cliente])

for cliente in CLIENTES:
    model.add_constraint(u[cliente] <= CAPACIDAD_CAMION)

In [157]:
model.set_time_limit(20)
solution = model.solve()

In [158]:
rutas = []
for i, cliente in enumerate(CLIENTES):
    if x[(0, cliente)].solution_value > 0.9:
        print(f"==== Ruta {i} ====")
        
        ruta = [0, cliente]
        r = "0 -> " + str(cliente) + " -> "

        c1 = cliente
        while x[(c1, 0)].solution_value < 0.9:
            c2 = next(c for c in CLIENTES if x[(c1, c)].solution_value > 0.9)
            r += str(c2) + " -> "
            ruta.append(c2)
            c1 = c2

        r += "0"
        ruta.append(0)

        print(r)
        rutas.append(ruta)

print("\n==== Arreglo de rutas ====")
print(rutas)


==== Ruta 1 ====
0 -> 2 -> 1 -> 4 -> 20 -> 0
==== Ruta 2 ====
0 -> 5 -> 11 -> 19 -> 12 -> 9 -> 0
==== Ruta 3 ====
0 -> 8 -> 6 -> 17 -> 10 -> 0
==== Ruta 4 ====
0 -> 13 -> 16 -> 7 -> 3 -> 0
==== Ruta 5 ====
0 -> 18 -> 14 -> 15 -> 0

==== Arreglo de rutas ====
[[0, 2, 1, 4, 20, 0], [0, 5, 11, 19, 12, 9, 0], [0, 8, 6, 17, 10, 0], [0, 13, 16, 7, 3, 0], [0, 18, 14, 15, 0]]


In [159]:
fig = go.Figure()

# RUTAS
for idx, ruta in enumerate(rutas):
    x = [COORDENADAS[nodo]["x"] for nodo in ruta]
    y = [COORDENADAS[nodo]["y"] for nodo in ruta]
    capacidad_ruta = sum(map(lambda x: DEMANDAS[x] if x != 0 else 0, ruta))
    distancia_ruta = sum([DISTANCIAS[(ruta[i], ruta[i + 1])] for i in range(len(ruta) - 1)])
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color=px.colors.qualitative.G10[idx % len(px.colors.qualitative.G10)], width=2, dash="solid"),
            name=f"Ruta {idx+1} - CAP: {capacidad_ruta} D: {distancia_ruta:.2f}",
        )
    )

# DEPÓSITO
fig.add_trace(
    go.Scatter(
        x=[COORDENADAS[0]["x"]],
        y=[COORDENADAS[0]["y"]],
        mode="markers",
        name="Depósito",
        marker=dict(color="red", size=15),
    )
)

# CLIENTES
fig.add_trace(
    go.Scatter(
        x=[coord["x"] for coord in list(COORDENADAS.values())[1:]],
        y=[coord["y"] for coord in list(COORDENADAS.values())[1:]],
        mode="markers",
        name="Clientes",
        marker=dict(color="black", size=10),
    )
)

for cliente in CLIENTES:
    coord_cliente_x = COORDENADAS[cliente]["x"]
    coord_cliente_y = COORDENADAS[cliente]["y"]
    fig.add_annotation(x=coord_cliente_x - 8, y=coord_cliente_y + 8, text=f"<b>C{cliente}</b>", showarrow=False)


fig.update_layout(
    title=f"RECORRIDO TOTAL: {solution.objective_value:.2f}",
    xaxis_title="X",
    yaxis_title="Y",
    xaxis=dict(range=[-10, N * 10 + 10]),
    yaxis=dict(range=[-10, N * 10 + 10]),
    width=800,
    height=600,
    template="ggplot2",
)

fig.show()

### Conjuntos
$$ \text{CLIENTES} = \{1, 2, \ldots, N\} $$
$$ \text{NODOS} = \{0\} \cup \text{CLIENTES} $$

### Parámetros
$$ \text{CAPACIDAD\_CAMION} = 70 $$
$$ \text{DEMANDA}_{c}: \text{Demanda del cliente } c \in \text{CLIENTES} $$
$$ \text{DISTANCIAS}_{n1,n2}: \text{Distancia euclidiana entre } n1 \in \text{NODOS} \text{ y } n2 \in \text{NODOS} $$

### Variables de Decisión
$$ x_{n1, n2} = \begin{cases} 
1 & \text{Si el camión viaja de } n1 \in \text{NODOS} \text{ al } n2 \in \text{NODOS} \\ 
0 & \text{d.l.c.} 
\end{cases} $$
$$ u_{n} \in \mathbb{Z}^{+} \text{: Carga acumulada del vehículo al llegar al nodo } n \in \text{NODOS} $$

### Función Objetivo
$$ \text{Minimizar } \sum_{n1 \in \text{NODOS}} \sum_{n2 \in \text{NODOS}} \text{DISTANCIAS}_{n1,n2} \cdot x_{n1, n2} $$

### Restricciones

1. **Prohibición de bucles:**
$$ \forall n \in \text{NODOS}: x_{n, n} = 0 $$

2. **Asignación única de salidas para cada cliente:**
$$ \forall c \in \text{CLIENTES}: \sum_{n \in \text{NODOS}, n \neq c} x_{c, n} = 1 $$

3. **Asignación única de entradas para cada cliente:**
$$ \forall c \in \text{CLIENTES}: \sum_{n \in \text{NODOS}, n \neq c} x_{n, c} = 1 $$

4. **Restricciones para evitar subtours:**
$$ \forall c1, c2 \in \text{CLIENTES}, c1 \neq c2: \text{Si } x_{c1, c2} = 1 \rightarrow u_{c2} =  u_{c1} + \text{DEMANDA}_{c2} $$

$$ \begin{cases} 
\forall c1, c2 \in \text{CLIENTES}, c1 \neq c2:  u_{c2} \leq u_{c1} + \text{DEMANDA}_{c2} + \mathcal{M} (1 - x_{c1, c2}) ❓\\
\forall c1, c2 \in \text{CLIENTES}, c1 \neq c2:  u_{c2} \geq u_{c1} + \text{DEMANDA}_{c2} - \mathcal{M} (1 - x_{c1, c2}) ✅
\end{cases} $$

5. **Demanda mínima en cada cliente:**
$$ \forall c \in \text{CLIENTES}: u_{c} \geq \text{DEMANDA}_{c} $$

6. **Demanda máxima permitida por el camión:**
$$ \forall c \in \text{CLIENTES}: u_{c} \leq \text{CAPACIDAD\_CAMION} $$

